In [1]:
import geemap
import ee
Map = geemap.Map()
Map

Enter verification code: 4/1AdQt8qgsh4tkwfk5qE8LG1hi1MH6SvGM1gV74J88KCVI0FoxRIXc2wZEiLs

Successfully saved authorization token.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [4]:
import os
# os.environ['PROJ_LIB'] = r'C:\Users\Administrator\Anaconda3\envs\py3.8\Library\share\proj'
# os.environ['GDAL_DATA'] = r'C:\Users\Administrator\Anaconda3\envs\py3.8\Library\share'

os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10809'
chain4 = geemap.shp_to_ee(r"G:\1_BeiJingUP\CarbonReport\Data\Climate4region\CClimate4Regions_4R.shp")
china = geemap.shp_to_ee(r"G:\1_BeiJingUP\CarbonReport\Data\Climate4region\CClimate4Regions_CHN.shp")

Map.addLayer(chain4, {}, 'CHN Climate Boundary')
Map

days=[31,28,31,30,31,30,31,31,30,31,30,31]

Map(bottom=1873.0, center=[38.47939467327645, 103.75488281250001], controls=(WidgetControl(options=['position'…

In [5]:
roi = china.merge(chain4)
import pandas as pd
df = pd.DataFrame([roi.getInfo()['features'][n]['properties'] for n in range(roi.size().getInfo())])
df

,Id,Name
0,0.0,NaN
1,NaN,Inland
2,NaN,QT
3,NaN,Temperate
4,NaN,Tropical


# PRCP

In [77]:
for year in range(1981,2021):
    yrImgC = ee.Image(0)
    print(year,end=' -> ')
    for month in range(1,13):
        # Change the numbers of the days in every month
        if (year//4==0):
            days[1]=29
        else:
            days[1]=28
        start_date = str(year) + '-' + str(month).zfill(2) + '-01'
        end_date = str(year) + '-' + str(month).zfill(2) + '-' + str(days[month-1])
        # print(start_date+' To '+end_date)
        era5_imgC = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select("total_precipitation").filterDate(start_date,end_date)
        # print('Numbers of Month Image: 'era5_imgC.size().getInfo())
        
        # Calculate the monthly value
        era5_month = era5_imgC.mean().multiply(1000*days[month-1]) # or use era5_imgC.first().multiply(days[month])
        
        # Calculate yearly value-------SUM
        yrImgC = yrImgC.add(era5_month)
    
    # Zonal Stati
    fes = geemap.zonal_statistics(yrImgC,roi,r"G:\1_BeiJingUP\CarbonReport\Data_0728\GEE_ERA5_total_precipitation.csv",
                                      "MEAN",return_fc=True,scale=11132)
    df[str(year)] = pd.DataFrame([fes.getInfo()['features'][n]['properties'] for n in range(roi.size().getInfo())])["mean"]
    # Map.addLayer(yrImgC, {}, 'Yearly PRCP')
    
# Save Outcome
df.to_excel(r"G:\1_BeiJingUP\CarbonReport\Data_0728\ERA5_total_precipitation.xlsx",index=None)
print('\n',df)

1981 -> Computing statistics ...
1982 -> Computing statistics ...
1983 -> Computing statistics ...
1984 -> Computing statistics ...
1985 -> Computing statistics ...
1986 -> Computing statistics ...
1987 -> Computing statistics ...
1988 -> Computing statistics ...
1989 -> Computing statistics ...
1990 -> Computing statistics ...
1991 -> Computing statistics ...
1992 -> Computing statistics ...
1993 -> Computing statistics ...
1994 -> Computing statistics ...
1995 -> Computing statistics ...
1996 -> Computing statistics ...
1997 -> Computing statistics ...
1998 -> Computing statistics ...
1999 -> Computing statistics ...
2000 -> Computing statistics ...
2001 -> Computing statistics ...
2002 -> Computing statistics ...
2003 -> Computing statistics ...
2004 -> Computing statistics ...
2005 -> Computing statistics ...
2006 -> Computing statistics ...
2007 -> Computing statistics ...
2008 -> Computing statistics ...
2009 -> Computing statistics ...
2010 -> Computing statistics ...
2011 -> Co

# TAVG

In [78]:
# for year in range(1981,2020):
#     era5 = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select("temperature_2m").filterDate(str(year),str(year+1)).mean()
#     fes = geemap.zonal_statistics(era5,roi,r"C:\Users\Administrator\Desktop\temperature_2m.csv",
#                         "MEAN",return_fc=True,scale=11132)
#     df[str(year)] = pd.DataFrame([fes.getInfo()['features'][n]['properties'] for n in range(roi.size().getInfo())])["mean"]
#     df.to_excel(r"D:\project\done\temperature_2m.xlsx",index=None)

for year in range(1981,2021):
    yrImgC = ee.Image(0)
    print(year,end=' -> ')
    for month in range(1,13):
        # Change the numbers of the days in every month
        if (year//4==0):
            days[1]=29
        else:
            days[1]=28
        start_date = str(year) + '-' + str(month).zfill(2) + '-01'
        end_date = str(year) + '-' + str(month).zfill(2) + '-' + str(days[month-1])
        # print(start_date+' To '+end_date)
        era5_imgC = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select("temperature_2m").filterDate(start_date,end_date)
        # print('Numbers of Month Image: 'era5_imgC.size().getInfo())
        
        # Calculate the monthly value
        era5_month = (era5_imgC.mean().subtract(273.15).multiply(days[month-1])) # or use era5_imgC.first().multiply(days[month])
        yrImgC = yrImgC.add(era5_month)
    
    # Calculate yearly value-------MEAN
    yrImgC = yrImgC.divide(sum(days))
    
    # Zonal Stati
    fes = geemap.zonal_statistics(yrImgC,roi,r"G:\1_BeiJingUP\CarbonReport\Data_0728\GEE_ERA5_temperature_2m.csv",
                                      "MEAN",return_fc=True,scale=11132)
    df[str(year)] = pd.DataFrame([fes.getInfo()['features'][n]['properties'] for n in range(roi.size().getInfo())])["mean"]
    # Map.addLayer(yrImgC, {}, 'Yearly PRCP')
    
# Save Outcome
df.to_excel(r"G:\1_BeiJingUP\CarbonReport\Data_0728\ERA5_temperature_2m.xlsx",index=None)
print('\n',df)

1981 -> Computing statistics ...
1982 -> Computing statistics ...
1983 -> Computing statistics ...
1984 -> Computing statistics ...
1985 -> Computing statistics ...
1986 -> Computing statistics ...
1987 -> Computing statistics ...
1988 -> Computing statistics ...
1989 -> Computing statistics ...
1990 -> Computing statistics ...
1991 -> Computing statistics ...
1992 -> Computing statistics ...
1993 -> Computing statistics ...
1994 -> Computing statistics ...
1995 -> Computing statistics ...
1996 -> Computing statistics ...
1997 -> Computing statistics ...
1998 -> Computing statistics ...
1999 -> Computing statistics ...
2000 -> Computing statistics ...
2001 -> Computing statistics ...
2002 -> Computing statistics ...
2003 -> Computing statistics ...
2004 -> Computing statistics ...
2005 -> Computing statistics ...
2006 -> Computing statistics ...
2007 -> Computing statistics ...
2008 -> Computing statistics ...
2009 -> Computing statistics ...
2010 -> Computing statistics ...
2011 -> Co

# SWRS

In [62]:
# for year in range(1981,2020):
#     era5 = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select("surface_solar_radiation_downwards").filterDate(str(year),str(year+1)).sum()
#     fes = geemap.zonal_statistics(era5,roi,r"C:\Users\Administrator\Desktop\surface_solar_radiation_downwards.csv",
#                         "MEAN",return_fc=True,scale=11132)
#     df[str(year)] = pd.DataFrame([fes.getInfo()['features'][n]['properties'] for n in range(roi.size().getInfo())])["mean"]
# df.to_excel(r"D:\project\done\surface_solar_radiation_downwards.xlsx",index=None)



1981 -> Computing statistics ...
    Id       Name          1981
0  0.0        NaN  1.661934e+07
1  NaN     Inland  1.698970e+07
2  NaN         QT  2.059251e+07
3  NaN  Temperate  1.458467e+07
4  NaN   Tropical  1.345263e+07


In [75]:
Map


Map(bottom=1171.0, center=[25.958044673317843, 116.45507812500001], controls=(WidgetControl(options=['position…